In [2]:
%pylab inline
import lasagne

Populating the interactive namespace from numpy and matplotlib


In [3]:
from __future__ import print_function

In [4]:
from fuel.transformers import ScaleAndShift, Cast, Flatten, Mapping
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme, ShuffledScheme

In [5]:
import numpy as np
import theano
import theano.tensor as T

In [25]:
import time
import sys

In [7]:
from fuel.datasets.cifar10 import CIFAR10

In [8]:
CIFAR10(("train",)).num_examples

50000

In [9]:
OFFSET = 40000

In [10]:
CIFAR10.default_transformers = (
    (ScaleAndShift, [2.0 / 255.0, -1], {'which_sources': 'features'}),
    (Cast, [np.float32], {'which_sources': 'features'}) )
#    (Flatten, [], {'which_sources': 'targets'}))
#    (Mapping, [lambda batch: (b.ravel() for b in batch)], {'which_sources': 'targets'}) )

mnist_train = CIFAR10(("train",), subset=slice(None,OFFSET))
#this stream will shuffle the MNIST set and return us batches of 100 examples
mnist_train_stream = DataStream.default_stream(
    mnist_train,
    iteration_scheme=ShuffledScheme(mnist_train.num_examples, 100))
                                               

In [11]:
print("The streams return batches containing %s" % (mnist_train_stream.sources,))

print("Each training batch consits of a tuple containing:")
for element in next(mnist_train_stream.get_epoch_iterator()):
    print(" - an array of size %s containing %s" % (element.shape, element.dtype))

The streams return batches containing (u'features', u'targets')
Each training batch consits of a tuple containing:
 - an array of size (100, 3, 32, 32) containing float32
 - an array of size (100, 1) containing uint8


In [12]:
mnist_validation = CIFAR10(("train",), subset=slice(OFFSET, None))

# We will use larger portions for testing and validation
# as these dont do a backward pass and reauire less RAM.
mnist_validation_stream = DataStream.default_stream(
    mnist_validation, iteration_scheme=SequentialScheme(mnist_validation.num_examples, 250))
mnist_test = CIFAR10(("test",))
mnist_test_stream = DataStream.default_stream(
    mnist_test, iteration_scheme=SequentialScheme(mnist_test.num_examples, 250))

In [13]:
print("Validation/test batches consits of tuples containing:")
for element in next(mnist_test_stream.get_epoch_iterator()):
    print(" - an array of size %s containing %s" % (element.shape, element.dtype))

Validation/test batches consits of tuples containing:
 - an array of size (250, 3, 32, 32) containing float32
 - an array of size (250, 1) containing uint8


In [14]:
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')

In [15]:
l_in = lasagne.layers.InputLayer(shape=(None, 3, 32, 32),
                                     input_var=input_var)

In [16]:
def network_combinator(in_layer, layers):
    for layer_def in layers:
        in_layer = layer_def[0](in_layer, **layer_def[1])
    return in_layer

In [17]:
def mlp(l_in):
    dropout = lambda p:  ( lasagne.layers.DropoutLayer, { 'p' : p } )
    dense_nonlinear = lambda u:  ( lasagne.layers.DenseLayer, {
                    'nonlinearity' : lasagne.nonlinearities.rectify,
                    'num_units' : u,
                })
    
    return network_combinator(l_in,[
            dropout(.2),
            dense_nonlinear(800),
            dropout(.5),
            dense_nonlinear(800),
            dropout(.5),
            ( lasagne.layers.DenseLayer, {
                    'num_units'  : 10,
                    'nonlinearity' : lasagne.nonlinearities.softmax })
        ])

In [18]:
def cnn(l_in):
    conv2d = lambda f, s: ( lasagne.layers.Conv2DLayer, {
            'num_filters' : f,
            'filter_size' : s,
            'nonlinearity' : lasagne.nonlinearities.rectify
            })
    
    
    maxpool2d = lambda s: ( lasagne.layers.MaxPool2DLayer, {
            'pool_size' : s
        })
    
    return network_combinator(l_in, [
            conv2d(32, (5,5)),
            maxpool2d((2,2)),
            conv2d(32, (5,5)),
            maxpool2d((2,2)),
            ( lasagne.layers.DropoutLayer, { 'p' : .5 } ),
            ( lasagne.layers.DenseLayer, {
                  'num_units' : 256  
                })
        ])

In [19]:
network = mlp(cnn(l_in))

In [20]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

In [21]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.01, momentum=0.9)

In [22]:
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
test_loss = test_loss.mean()
# As a bonus, also create an expression for the classification accuracy:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

In [23]:
train_fn = theano.function([input_var, target_var], loss, updates=updates)

# Compile a second function computing the validation loss and accuracy:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [26]:
def train(train_stream, validation_stream, num_epochs, f=sys.stdout):
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for inputs, targets in train_stream.get_epoch_iterator():
            train_err += train_fn(inputs, targets.ravel())
            train_batches += 1

        # And a full pass over the validation data:
        val_err = 0
        val_acc = 0
        val_batches = 0
        for inputs, targets in validation_stream.get_epoch_iterator():
            err, acc = val_fn(inputs, targets.ravel())
            val_err += err
            val_acc += acc
            val_batches += 1

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time),
            file=f)
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches)
              , file=f)
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches)
              , file=f)
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100)
              , file=f)
        f.flush()

In [27]:
train(mnist_train_stream, mnist_validation_stream, 1)

Epoch 1 of 1 took 62.518s
  training loss:		1.992430
  validation loss:		1.703029
  validation accuracy:		37.49 %


In [ ]:
f = open('workfile', 'w', 0)
try:
    train(mnist_train_stream, mnist_validation_stream, 100, f = f)
finally:
    f.close()

In [30]:
val_acc = 0
batches = 0
for inputs, targets in mnist_test_stream.get_epoch_iterator():
    _, acc = val_fn(inputs, targets.ravel())
    val_acc += acc
    batches += 1
print("  train accuracy:\t\t{:.2f} %".format(
    val_acc / batches * 100))

  train accuracy:		76.47 %


In [31]:
import pickle
f = open('parameters', 'w')
try:
    pickle.dump(network, f)
finally:
    f.close()

In [32]:
import numpy
numpy.save('nparameters', network)